In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import mean_squared_error
import math

In [2]:
train = pd.read_csv('FinalTrain.csv',index_col=0)

In [3]:
bool_cols = [col for col in train if train[col].dropna().value_counts().index.isin([0,1]).all()]

In [4]:
binary = pd.Index(bool_cols)

In [6]:
continous_variables = train.columns.difference(binary)
#continous_variables = continous_variables.drop('CloudCover')

In [7]:
ordinal_variables = pd.Index(['CloudCover'])

# Without Double Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.metrics import r2_score

In [ ]:
train.head()

In [ ]:
train_data, test_data = model_selection.train_test_split(train, test_size = 0.3, random_state=1234)

In [ ]:
NumberOfSales = test_data['NumberOfSales']

In [ ]:
r2_num_of_sales = []
models = []

target = train_data['NumberOfSales']
variables = train_data.drop(labels='NumberOfCustomers',axis=1)
variables = variables.drop(labels='NumberOfSales', axis=1)

for i in range(100,151,25):

    
    m1 = RandomForestRegressor(oob_score=True,random_state=1234) 
    m1.set_params(n_estimators=i)
    m1.fit(variables, target)
    
    r2_num_of_sales.append(m1.oob_score_)

In [ ]:
r2

In [ ]:
test_data = test_data.drop(labels='NumberOfSales',axis=1)
test_data = test_data.drop(labels='NumberOfCustomers',axis=1)

Check r2 and mse also on the test for each model built

In [ ]:
r2_on_test_set = []
mse_on_test_set = []
for model in models:
    yp = model.predict(test_data)
    r2_on_test_set.append(r2_score(NumberOfSales,yp))
    mse_on_test_set.append(mean_squared_error(NumberOfSales,yp))

In [ ]:
r2_on_test_set

In [ ]:
np.sqrt(mse_on_test_set)

In [ ]:
NumberOfSales.describe()

# Double Regression

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.metrics import r2_score

In [5]:
train = pd.read_csv('FinalTrain.csv',index_col=0)
train.head()

,component 1,component 2,component 3,component 5,component 6,component 7,component 8,component 9,component 10,HasPromotions,...,May,June,September,October,November,December,Fog,Snow,NumberOfCustomers,NumberOfSales
0,3.381430,1.782698,-1.783757,-1.810411,-0.243794,0.433565,1.163843,-1.407220,-0.700026,0,...,0,0,0,0,0,0,0,1,495,2.266354
1,2.332788,2.793018,-0.828153,-0.597817,-0.693162,0.771066,1.181362,-1.565924,-0.065226,0,...,0,0,0,0,0,0,0,1,608,2.302695
2,2.238024,2.548715,0.108930,-0.898286,-0.640268,0.550553,1.165865,-1.488746,-0.003380,0,...,0,0,0,0,0,0,0,0,665,2.304995
3,1.768849,4.284677,0.925293,-0.049667,-0.584788,-0.535921,1.217520,-0.808803,0.639240,0,...,0,0,0,0,0,0,0,0,630,2.290064
4,1.379854,3.592518,1.562039,0.092145,-1.278158,-0.375772,1.574675,-0.333073,-0.517771,1,...,0,0,0,0,0,0,0,0,763,2.324482


## 50

In [15]:
train_data, test_data = model_selection.train_test_split(train, test_size = 0.3, random_state=1234)

In [16]:
NumberOfCust = test_data['NumberOfCustomers']
NumberOfSales = test_data['NumberOfSales']

In [17]:
target = train_data['NumberOfCustomers']
variables = train_data.drop(labels='NumberOfCustomers',axis=1)
variables = variables.drop(labels='NumberOfSales', axis=1)

rfr = RandomForestRegressor(random_state=1234) 
rfr.set_params(n_estimators=50)
rfr.fit(variables, target)


cust = rfr.predict(variables)
target = train_data['NumberOfSales']
variables['NumberOfCustomers'] = cust

m1 = RandomForestRegressor(oob_score=True,random_state=1234) 
m1.set_params(n_estimators=50)
m1.fit(variables, target)
m1.oob_score_  

0.7822755473217412

In [18]:
test_data = test_data.drop(labels='NumberOfSales',axis=1)

In [20]:
yp = m1.predict(test_data)
print(str(r2_score(NumberOfSales,yp)))
print(str(mean_squared_error(NumberOfSales,yp)))

-0.3383220985816786
0.0031001238391371276


## 75

In [6]:
train_data, test_data = model_selection.train_test_split(train, test_size = 0.3, random_state=1234)

In [7]:
NumberOfCust = test_data['NumberOfCustomers']
NumberOfSales = test_data['NumberOfSales']

In [8]:
target = train_data['NumberOfCustomers']
variables = train_data.drop(labels='NumberOfCustomers',axis=1)
variables = variables.drop(labels='NumberOfSales', axis=1)

rfr = RandomForestRegressor(random_state=1234) 
rfr.set_params(n_estimators=75)
rfr.fit(variables, target)


cust = rfr.predict(variables)
target = train_data['NumberOfSales']
variables['NumberOfCustomers'] = cust

m1 = RandomForestRegressor(oob_score=True,random_state=1234) 
m1.set_params(n_estimators=75)
m1.fit(variables, target)
m1.oob_score_  

0.775395117962749

In [9]:
test_data = test_data.drop(labels='NumberOfSales',axis=1)

In [11]:
yp = m1.predict(test_data)
print(str(r2_score(NumberOfSales,yp)))
print(str(mean_squared_error(NumberOfSales,yp)))

-0.367840578599403
0.0031685012078550333


## 100

In [10]:
train_data, test_data = model_selection.train_test_split(train, test_size = 0.3, random_state=1234)

In [11]:
NumberOfCust = test_data['NumberOfCustomers']
NumberOfSales = test_data['NumberOfSales']

In [12]:
target = train_data['NumberOfCustomers']
variables = train_data.drop(labels='NumberOfCustomers',axis=1)
variables = variables.drop(labels='NumberOfSales', axis=1)
    
m1 = RandomForestRegressor(oob_score=True,random_state=1234) 
m1.set_params(n_estimators=100)
m1.fit(variables, target)

    
cust = m1.predict(variables)
target = train_data['NumberOfSales']
variables['NumberOfCustomers'] = cust
    
m2 = RandomForestRegressor(oob_score=True,random_state=1234)
m2.set_params(n_estimators=100)
m2.fit(variables, target)
    
m2.oob_score_

In [16]:
test_data = test_data.drop(labels='NumberOfSales',axis=1)

In [17]:
yp = m2.predict(test_data)
print(str(r2_score(NumberOfSales,yp)))
print(str(mean_squared_error(NumberOfSales,yp)))

## 125

In [91]:
train_data, test_data = model_selection.train_test_split(train, test_size = 0.3, random_state=1234)

In [92]:
NumberOfCust = test_data['NumberOfCustomers']
NumberOfSales = test_data['NumberOfSales']

In [93]:
target = train_data['NumberOfCustomers']
variables = train_data.drop(labels='NumberOfCustomers',axis=1)
variables = variables.drop(labels='NumberOfSales', axis=1)
    
m1 = RandomForestRegressor(oob_score=True,random_state=1234) 
m1.set_params(n_estimators=125)
m1.fit(variables, target)
    
    
cust = m1.predict(variables)
target = train_data['NumberOfSales']
variables['NumberOfCustomers'] = cust
    
m2 = RandomForestRegressor(oob_score=True,random_state=1234)
m2.set_params(n_estimators=125)
m2.fit(variables, target)
m2.oob_score_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=125, n_jobs=1,
           oob_score=True, random_state=1234, verbose=0, warm_start=False)

In [ ]:
test_data = test_data.drop(labels='NumberOfSales',axis=1)

In [94]:
yp = m2.predict(test_data)
print(str(r2_score(NumberOfSales,yp)))
print(str(mean_squared_error(NumberOfSales,yp)))

## 150

In [97]:
train_data, test_data = model_selection.train_test_split(train, test_size = 0.3, random_state=1234)

In [98]:
NumberOfCust = test_data['NumberOfCustomers']
NumberOfSales = test_data['NumberOfSales']

In [99]:
target = train_data['NumberOfCustomers']
variables = train_data.drop(labels='NumberOfCustomers',axis=1)
variables = variables.drop(labels='NumberOfSales', axis=1)
    
m1 = RandomForestRegressor(oob_score=True,random_state=1234) 
m1.set_params(n_estimators=150)
m1.fit(variables, target)
    
    
cust = m1.predict(variables)
target = train_data['NumberOfSales']
variables['NumberOfCustomers'] = cust
    
m2 = RandomForestRegressor(oob_score=True,random_state=1234)
m2.set_params(n_estimators=150)
m2.fit(variables, target)
m2.oob_score_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=1,
           oob_score=True, random_state=1234, verbose=0, warm_start=False)

In [ ]:
test_data = test_data.drop(labels='NumberOfSales',axis=1)

In [102]:
yp = m2.predict(test_data)
print(str(r2_score(NumberOfSales,yp)))
print(str(mean_squared_error(NumberOfSales,yp)))

0.726603625418111
0.0007496384340665233


# Trying to build a regressor model for each type of store

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import mean_squared_error
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.metrics import r2_score

In [2]:
train = pd.read_csv('FinalTrain.csv',index_col=0)

In [3]:
train_data, test_data = model_selection.train_test_split(train, test_size = 0.3, random_state=1234)

In [4]:
target = train_data['NumberOfCustomers']
variables = train_data.drop(labels='NumberOfCustomers',axis=1)
variables = variables.drop(labels='NumberOfSales', axis=1)

var_1 = variables.loc[variables['HyperMarket'] == 1]
var_2 = variables.loc[variables['SuperMarket'] == 1]
var_3 = variables.loc[variables['StandardMarket'] == 1]
var_4 = variables.loc[variables['ShoppingCenter'] == 1]

tar_1 = train_data.loc[train_data['HyperMarket'] == 1]['NumberOfCustomers']
tar_2 = train_data.loc[train_data['SuperMarket'] == 1]['NumberOfCustomers']
tar_3 = train_data.loc[train_data['StandardMarket'] == 1]['NumberOfCustomers']
tar_4 = train_data.loc[train_data['ShoppingCenter'] == 1]['NumberOfCustomers']

In [5]:
c1 = RandomForestRegressor(random_state=1234) 
c2 = RandomForestRegressor(random_state=1234) 
c3 = RandomForestRegressor(random_state=1234) 
c4 = RandomForestRegressor(random_state=1234) 
c1.set_params(n_estimators=125)
c2.set_params(n_estimators=125)
c3.set_params(n_estimators=125)
c4.set_params(n_estimators=125)
c1.fit(var_1, tar_1)
c2.fit(var_2, tar_2)
c3.fit(var_3, tar_3)
c4.fit(var_4, tar_4)

cust1 = c1.predict(var_1)
cust2 = c2.predict(var_2)
cust3 = c3.predict(var_3)
cust4 = c4.predict(var_4)

var_1['NumberOfCustomers'] = cust1
var_2['NumberOfCustomers'] = cust2
var_3['NumberOfCustomers'] = cust3
var_4['NumberOfCustomers'] = cust4

tar_1 = train_data.loc[train_data['HyperMarket'] == 1]['NumberOfSales']
tar_2 = train_data.loc[train_data['SuperMarket'] == 1]['NumberOfSales']
tar_3 = train_data.loc[train_data['StandardMarket'] == 1]['NumberOfSales']
tar_4 = train_data.loc[train_data['ShoppingCenter'] == 1]['NumberOfSales']



s1 = RandomForestRegressor(oob_score=True,random_state=1234) 
s2 = RandomForestRegressor(oob_score=True,random_state=1234) 
s3 = RandomForestRegressor(oob_score=True,random_state=1234) 
s4 = RandomForestRegressor(oob_score=True,random_state=1234)
s1.set_params(n_estimators=125)
s2.set_params(n_estimators=125)
s3.set_params(n_estimators=125)
s4.set_params(n_estimators=125)
s1.fit(var_1, tar_1)
s2.fit(var_2, tar_2)
s3.fit(var_3, tar_3)
s4.fit(var_4, tar_4)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\sit

NameError: name 'S4' is not defined

In [7]:
test_1 =  test_data.loc[test_data['HyperMarket'] == 1]
test_2 =  test_data.loc[test_data['SuperMarket'] == 1]
test_3 =  test_data.loc[test_data['StandardMarket'] == 1]
test_4 =  test_data.loc[test_data['ShoppingCenter'] == 1]

In [8]:
sales1 = test_1['NumberOfSales']
sales2 = test_2['NumberOfSales']
sales3 = test_3['NumberOfSales']
sales4 = test_4['NumberOfSales']

In [9]:
test_1 = test_1.drop(labels='NumberOfSales',axis=1)
test_2 = test_2.drop(labels='NumberOfSales',axis=1)
test_3 = test_3.drop(labels='NumberOfSales',axis=1)
test_4 = test_4.drop(labels='NumberOfSales',axis=1)

In [10]:
yp = s1.predict(test_1)
print(str(r2_score(sales1,yp)))
print(str(mean_squared_error(sales1,yp)))
yp = s2.predict(test_2)
print(str(r2_score(sales2,yp)))
print(str(mean_squared_error(sales2,yp)))
yp = s3.predict(test_3)
print(str(r2_score(sales3,yp)))
print(str(mean_squared_error(sales3,yp)))
yp = s4.predict(test_4)
print(str(r2_score(sales4,yp)))
print(str(mean_squared_error(sales4,yp)))

0.6390843396667892
0.00098727583135878
0.6895962886483329
0.0006364810986711153
0.6660428748543659
0.0005042486842456487
0.9406875528177541
0.00012673677152988746
